In [ ]:
'''
The leg is structured as follows:
(trip_id, departure_node, departure_time, arrival_node, arrival_time, route_id, service_id)

trip_id = leg.iloc[0]
departure_node = leg[1]
departure_time = leg[2]
arrival_node = leg[3]
arrival_time = leg[4]
route_id = leg[5]
service_id = leg[6]
'''

'\nThe leg is structured as follows:\n(trip_id, departure_node, departure_time, arrival_node, arrival_time, route_id, service_id)\n\ntrip_id = leg.iloc[0]\ndeparture_node = leg[1]\ndeparture_time = leg[2]\narrival_node = leg[3]\narrival_time = leg[4]\nroute_id = leg[5]\nservice_id = leg[6]\n'

In [2]:
from datetime import datetime, timedelta
import pandas as pd
import math
import scipy.stats as stats
from data_preparation import prepare_data,import_data
pd.set_option('display.max_colwidth', None)

In [3]:
agency_df, stops_df, routes_df, trips_df, stop_times_df, calendar_df,calendar_dates_df = import_data()
legs_df = prepare_data(stops_df,trips_df,stop_times_df)

In [4]:
print(legs_df.head())

0                             (1.TA.1-1-j24-1.1.H, Bruck/Mur Bahnhof, 01:30:00, Kapfenberg Bahnhof, 01:34:00, 1-1-j24-1, TA+a4530)
1                     (1.TA.1-1-j24-1.1.H, Kapfenberg Bahnhof, 01:35:00, Kapfenberg Fachhochschule, 01:37:00, 1-1-j24-1, TA+a4530)
2          (1.TA.1-1-j24-1.1.H, Kapfenberg Fachhochschule, 01:38:00, St.Marein-St.Lorenzen Bahnhof, 01:41:00, 1-1-j24-1, TA+a4530)
3    (1.TA.1-1-j24-1.1.H, St.Marein-St.Lorenzen Bahnhof, 01:41:00, Allerheiligen-Mürzhofen Bahnhof, 01:44:00, 1-1-j24-1, TA+a4530)
4                 (1.TA.1-1-j24-1.1.H, Allerheiligen-Mürzhofen Bahnhof, 01:44:00, Kindberg Bahnhof, 01:47:00, 1-1-j24-1, TA+a4530)
Name: leg, dtype: object


In [5]:
# Function to calculate probability of successful transfer between two subsequent legs
def calculate_transfer_probability(prev_leg: pd.Series, next_leg: pd.Series) -> float:
    if prev_leg[0] == next_leg[0]:
        return 1
    else:
        prev_arrival_time = datetime.strptime(prev_leg[4], "%H:%M:%S")
        next_departure_time = datetime.strptime(next_leg[2], "%H:%M:%S")
        
        # Calculate transfer time in minutes
        transfer_time = (next_departure_time - prev_arrival_time).total_seconds() / 60
        if transfer_time < 2:
            return 0
        #If tripId = next tripId , then probability = 1 since the traveler remains on the same line
        else:
            return stats.gamma.cdf(transfer_time, a=2, scale=2)
            #return 0.5


In [6]:
# i hope it is cumulative distribution
def calculate_cumulative_probability(itinerary) -> list[float]:
    cumulative_probabilities = [1] 
    for i in range(len(itinerary) - 1):
        prev_leg = itinerary[i]
        next_leg = itinerary[i+1]

        transfer_prob = calculate_transfer_probability(prev_leg, next_leg)
        cumulative_probabilities.append(transfer_prob)
    return cumulative_probabilities
  

In [7]:
# if there is a chance to have all succesful transfers, is the arrival time in time budget
def calculate_arrival_probability(itinerary, start_time, time_budget) -> int:
    # Check if all previous transfers were successful
    if math.prod(calculate_cumulative_probability(itinerary)) > 0:  # Only proceed if all transfers are successful( naka can be possibly made)
        # Calculate the actual arrival time at the final leg
        destination_leg = itinerary[-1]
        destination_arrival_time = destination_leg[4]
        
        start_time = datetime.strptime(start_time, "%H:%M:%S")
        destination_arrival_time = datetime.strptime(destination_arrival_time, "%H:%M:%S")

        total_travel_time = destination_arrival_time - start_time

        # Check if total travel time is within the budget
        if total_travel_time <= time_budget:
            return 1  # Probability of 1 if arrival is within the time budget
        else:
            return 0  # Probability of 0 if arrival is beyond the time budget
    else:
        return 0  # Probability of 0 if any prior transfer was unsuccessful

In [8]:
def primary_itinerary_reliability(itinerary, start_time, time_budget) -> float:
    
    cumulative_probabilities = calculate_cumulative_probability(itinerary)
    product_of_probabilities= math.prod(cumulative_probabilities)
    arrival_probability = calculate_arrival_probability(itinerary,start_time,time_budget)

    reliability = arrival_probability * product_of_probabilities
    return reliability

To change this

In [9]:
#im not sure how the "backup" will look like (like is it starting from the origin or from the missed transfer point)
# backup = (leg of the prim it where is transfer, [sequence of legs of backup starting from the next after transfer], reliability, arrival time)
#backup_legs = [list of backups excluding prim]
def backup_itinerary_reliability(itinerary, backup:tuple, start_time, time_budget) -> float:
    
    #change the format
    backup_itinerary = backup[1][:]
    arrival_probability = calculate_arrival_probability(backup_itinerary, start_time, time_budget)
    cumulative_probabilities = calculate_cumulative_probability(backup_itinerary)
    product_of_probabilities= math.prod(cumulative_probabilities)
    
    #some adj
    #primary_reliability = primary_itinerary_reliability(itinerary,start_time,time_budget)

    transfer_leg = backup[0] #leg of prim itn.
    transfer_point = transfer_leg[3]
    initial_transfer_prob = 1 # Default value in case no missed transfer is identified
    backup_transfer_prob = calculate_transfer_probability(transfer_leg,backup_itinerary[0])
    #but what if backup not to the last leg????
    for idx, leg in enumerate(itinerary[:-1]):  # Exclude last item since we're accessing i+1
        #basically comparing if it is the same leg
        if leg[3] == transfer_leg[3]:
            prev_leg = itinerary[idx]
            missed_leg = itinerary[idx + 1]
            initial_transfer_prob = calculate_transfer_probability(prev_leg,missed_leg)
            primary_itinerary_rel_before_transfer = primary_itinerary_reliability(itinerary[:idx+1],start_time,time_budget)
            #print("init_tr",initial_transfer_prob)
            break
    
    backup_reliability = arrival_probability * product_of_probabilities * (1 - initial_transfer_prob) *primary_itinerary_rel_before_transfer
    #print("b_rel",backup_reliability)
    return backup_reliability


In [10]:
#Reliability of a complete itinerary 
#Backups will be set of backups of prim itineary
# backup itineary= (leg of the prim it where is transfer, [sequence of legs of backup starting from the next after transfer], reliability, arrival time)
def itinerary_reliability(itinerary : list[pd.Series],Backups : list[tuple], start_time:str, time_budget: timedelta) -> float:
    primary_reliability = primary_itinerary_reliability(itinerary,start_time,time_budget)
    #print("prim",primary_reliability)
    if primary_reliability > 0 :
        added_reliability = 0
        for backup in Backups:
            #this will be fucked up, as we dont know yet how to make backups ( like on what format)
            backup_reliability = backup_itinerary_reliability(itinerary,backup,start_time,time_budget)
            added_reliability += backup_reliability
        #added_reliability = min(added_reliability, 1-primary_reliability)
        complete_reliability = primary_reliability + added_reliability
        return complete_reliability
    else:
        return 0.0

In [11]:
def get_available_service_ids(start_date):
    # Convert start_date to datetime and then to integer-like string (YYYYMMDD) without dashes
    start_date_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    start_date_str = start_date_datetime.strftime("%Y%m%d")
    weekday = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"][start_date_datetime.weekday()]

    available_service_ids = []


    # Step 2: Check for regular service in calendar
    for _, service in calendar_df.iterrows():
        service_id = service["service_id"]
        #print(service_id)
        # Check if date is within the start_date and end_date range
        if int(service["start_date"]) <= int(start_date_str) <= int(service["end_date"]):
            # Check if the service operates on this weekday
            if service[weekday] == 1:
                #print(service_id)
                if service_id not in available_service_ids:
                    #print(service_id)
                    available_service_ids.append(service_id)
                                
    # Step 1: Check for exceptions in calendar_dates
    exceptions = calendar_dates_df[calendar_dates_df["date"] == int(start_date_str)]
    
    # Go through each service_id to check for exceptions
    for _, exception in exceptions.iterrows():
        service_id = exception["service_id"]
        if exception["exception_type"] == 2:  # Service is added as an exception
            if service_id not in available_service_ids:
                available_service_ids.append(service_id)
                
        elif exception["exception_type"] == 1:  # Service is removed as an exception
            if service_id in available_service_ids:
                available_service_ids.remove(service_id)
                
    

    return available_service_ids

In [12]:
def filter_network(start_time,start_date,time_budget):
    available_sevices = get_available_service_ids(start_date)

    filtered_network= []
    start_time = datetime.strptime(start_time, "%H:%M:%S")
    end_time = start_time + time_budget
    print(end_time)
    for row in legs_df:
        leg_departure_time = datetime.strptime(row[2],"%H:%M:%S")
        leg_arrival_time = datetime.strptime(row[4],"%H:%M:%S")
        if leg_departure_time >= start_time and leg_departure_time<= end_time and leg_arrival_time <= end_time:
            service_id = row[6]
            if service_id in available_sevices:
                filtered_network.append(row)
    print("Network",len(filtered_network))
    return filtered_network

In [13]:
#not sure if here we should add time dependancy
def search_adjecent_legs(node_id,arrival_time,filtered_legs):
    arrival_time = datetime.strptime(arrival_time, "%H:%M:%S")
    adjecent_legs = []
    for i in range(0,len(filtered_legs)):
        leg = filtered_legs[i]
        leg_departure_time = datetime.strptime(leg[2],"%H:%M:%S")
        if leg[1] == node_id and leg_departure_time >= arrival_time :
            adjecent_legs.append(leg)
    return adjecent_legs

In [14]:
# need to add to arrival probability
def travel_time(itinerary, start_time):
    destination_leg = itinerary[-1]
    destination_arrival_time = destination_leg[4]
        
    start_time = datetime.strptime(start_time, "%H:%M:%S")
    destination_arrival_time = datetime.strptime(destination_arrival_time, "%H:%M:%S")

    total_travel_time = destination_arrival_time - start_time
    return total_travel_time

In [15]:
def is_transfer(itinerary) -> bool:
    prev_leg = itinerary[-2]
    next_leg = itinerary[-1]
    if prev_leg[0] == next_leg[0]:
        return False
    else:
        return True

In [16]:
def find_min_index(LIST:list):
    """Find the index of the itinerary with the shortest duration."""
    #adjust for bachup
    min_index = 0
    min_value =  LIST[0][2] 
    for i in range(1, len(LIST)):
        current_value = LIST[i][2]
        if current_value < min_value:
            min_value = current_value
            min_index = i

    return min_index

In [17]:
def update_visited_stops(visited_stops, leg):
    """Update the visited stops dictionary with a new leg."""
    destination_stop = leg[1]
    arrival_time = datetime.strptime(leg[4], "%H:%M:%S")
    trip_id = leg[0]
    visited_stops[destination_stop] = (arrival_time, trip_id)

In [18]:
def check_and_update_mrib(shortest_path, MRIB_reliability, MRIB, start_time, time_budget):
    """Check and update the most reliable path (MRIB) if applicable."""
    Backups = shortest_path[4][:]
    rel = itinerary_reliability(shortest_path[0], Backups, start_time, time_budget)
    if rel > MRIB_reliability:
        print("New most reliable path found")
        MRIB_reliability = rel
        MRIB = shortest_path
    return MRIB_reliability, MRIB

In [19]:
def can_visit_leg(leg, visited_stops, shortest_path_trip_id, destination_node):


    arrival_stop = leg[3]  # Get destination stop ID
    arrival_time = datetime.strptime(leg[4], "%H:%M:%S")  # Arrival time
    trip_id = leg[0]  # Assuming trip_id is at index 0
    if destination_node != arrival_stop:
        if arrival_stop in visited_stops:
            visited_time, visited_trip_id = visited_stops[arrival_stop]
            if trip_id == shortest_path_trip_id:
                if arrival_time <= visited_time:
                    visited_stops[arrival_stop] = (arrival_time, trip_id)
                return True
            else : 
                if arrival_time > visited_time:
                    return False
                elif arrival_time == visited_time and trip_id == visited_trip_id:
                    return True
                elif arrival_time == visited_time:
                    return False
                else:
                    visited_stops[arrival_stop] = (arrival_time, trip_id)
                    return True

        else:
            visited_stops[arrival_stop] = (arrival_time, trip_id)
            return True
    return True

In [20]:
def origin_node_filtering(origin_node: int, start_time: str, time_budget: timedelta, legs: list):
    """
    Filters legs to ensure departure from the origin occurs within the first 25% of the time budget.

    Parameters:
    origin_node (int): The origin stop/node.
    start_time (str): The starting time in "%H:%M:%S" format.
    time_budget (timedelta): The total allowed time budget.
    legs (list): The list of available legs to filter.

    Returns:
    list: Filtered legs meeting the departure condition.
    """
    
    # Calculate the 20% time limit from the time budget
    max_departure_time = datetime.strptime(start_time, "%H:%M:%S") + time_budget * 0.25

    # Filter legs based on departure time
    filtered_legs = []
    for leg in legs:
        departure_time = datetime.strptime(leg[2], "%H:%M:%S")  # Assuming departure time is stored in this location
        if departure_time <= max_departure_time:
            filtered_legs.append(leg)

    return filtered_legs


In [21]:
def transform_route_info(MRIB,MRIB_reliability):
    primary_itinerary = MRIB[0]  # Primary itinerary (first item in best_result_fast)
    reliability = MRIB_reliability  # Reliability score
    duration = MRIB[2]  # Duration of the trip
    arrival_time = MRIB[3]  # Arrival time
    backups = MRIB[4]  # Backup routes
    
    # Initialize the grouped_routes list
    grouped_routes = []
    last_route = None  # To track the previous route for grouping

    # Step 1: Process the primary itinerary
    for i in range(len(primary_itinerary)):
        current_stop = primary_itinerary[i][1]
        route_id = primary_itinerary[i][5]
        departure_time = primary_itinerary[i][2]
        arrival_time = primary_itinerary[i][4]
        next_stop = primary_itinerary[i][3]

        # Grouping the routes based on the route ID
        if route_id == last_route:
            grouped_routes[-1]["stops"].append((next_stop, arrival_time))
        else:
            grouped_routes.append({
                "route_id": route_id,
                "start_stop": current_stop,
                "departure_time": departure_time,
                "stops": [(next_stop, arrival_time)]
            })
        last_route = route_id

    # Step 2: Print the grouped routes for the primary itinerary
    for segment in grouped_routes:
        start = segment["start_stop"]
        dep_time = segment["departure_time"]
        route = segment["route_id"]
        stops = " → ".join([f"{stop} (Ankunft: {arr})" for stop, arr in segment["stops"]])
        print(f"  🚆 {start} (Abfahrt: {dep_time}) → {stops} mit Linie {route}")

    # Step 3: Print additional details
    print(f"\n🎯 Endstation: {MRIB[-2]} (Ankunft: {arrival_time})")
    print(f"🔹 Gesamt-Zuverlässigkeit der Route: {reliability:.2f}\n")
    
    # Step 4: Process and print the backup routes
    if backups:
        print("🔄 Backups:")
        for backup in backups:
            primary_path = backup[0]
            backup_path = backup[1]
            backup_reliability = backup[2]
            grouped_backup_routes = []
            last_backup_route = None

            for i in range(len(backup_path)):
                current_stop = backup_path[i][1]
                route_id = backup_path[i][5]
                departure_time = backup_path[i][2]
                arrival_time = backup_path[i][4]
                next_stop = backup_path[i][3]

                # Group backup routes by route ID
                if route_id == last_backup_route:
                    grouped_backup_routes[-1]["stops"].append((next_stop, arrival_time))
                else:
                    grouped_backup_routes.append({
                        "route_id": route_id,
                        "start_stop": current_stop,
                        "departure_time": departure_time,
                        "stops": [(next_stop, arrival_time)]
                    })
                last_backup_route = route_id

            # Print backup route segments
            for segment in grouped_backup_routes:
                start = segment["start_stop"]
                dep_time = segment["departure_time"]
                route = segment["route_id"]
                stops = " → ".join([f"{stop} (Ankunft: {arr})" for stop, arr in segment["stops"]])
                print(f"  🚆 {start} (Abfahrt: {dep_time}) → {stops} mit Linie {route}")

            print(f"🔹 Gesamt-Zuverlässigkeit der Backup-Route: {backup_reliability:.2f}\n")
            


In [22]:
def backup_search(shortest_path,shortest_next_itinerary,destination_node,start_time,time_budget,filtered_legs):

    print("New backup search")
    #Initial Setup
    MRB_reliability = 0
    MRB = None
    LIST_Backups = []
    
   
    transfer_leg = shortest_path[0][-1]
    transfer_point = transfer_leg[3]
    primary_itinerary = shortest_next_itinerary[0]
    missed_leg_dep_time = datetime.strptime(primary_itinerary[-1][2], "%H:%M:%S") 


    passed_stops_b =[]
    for leg in primary_itinerary[:-1]:
        passed_stops_b.append(leg[3])
    passed_stops_b.append(primary_itinerary[0][1])
    
    adjecent_legs = search_adjecent_legs(transfer_point,primary_itinerary[-1][2],filtered_legs)

    adjecent_legs = [leg for leg in adjecent_legs if leg[3] not in passed_stops_b]

    adjecent_legs = [
            leg for leg in adjecent_legs 
            if datetime.strptime(leg[2], "%H:%M:%S") >= (missed_leg_dep_time + timedelta(minutes=2)) 
        ]
    
    #Transforming backups
    for leg in adjecent_legs: 
        backup = (transfer_leg,[leg])
        b_reliability = backup_itinerary_reliability(primary_itinerary,backup,start_time,time_budget)
        backup_full = (transfer_leg,[leg],b_reliability)
        b_duration = travel_time([leg],start_time)
        if b_reliability > 0 and timedelta(seconds=0) < b_duration <= time_budget: # check initially so we dont waste time later
                LIST_Backups.append(backup_full)
                

    #Backup loop
    while len(LIST_Backups) > 0: 
        min_index_b = 0
        min_value_b =  LIST_Backups[0][1][-1][4]# the arrival time of the last leg of backup
        for i in range(1, len(LIST_Backups)):
            current_value_b = LIST_Backups[i][1][-1][4]
            if current_value_b < min_value_b:
                min_value_b = current_value_b
                min_index_b = i
    	
        shortest_backup = LIST_Backups.pop(min_index_b)

        for leg in shortest_backup[1]:
            destination_stop = leg[3]
            passed_stops_b.append(destination_stop)

        b_tail = shortest_backup[1][-1]

        if b_tail[3] == destination_node:
            rel = backup_itinerary_reliability(shortest_next_itinerary[0],shortest_backup,start_time,time_budget)
            if rel > MRB_reliability:
                MRB_reliability = rel
                MRB = shortest_backup
        else:
            
            next_legs_b = search_adjecent_legs(b_tail[3],b_tail[4],filtered_legs)
            #next_legs_b_filtered = filter_adjecent_legs(next_legs_b,b_tail.iloc[0]) 
            
            next_legs_b = [leg for leg in next_legs_b if leg[3] not in passed_stops_b]
            
            next_legs_b = [
            leg for leg in next_legs_b 
            if leg[0] == b_tail[0] or datetime.strptime(leg[2], "%H:%M:%S") >= (datetime.strptime(b_tail[4], "%H:%M:%S") + timedelta(minutes=2))
        ]
            
            for leg in next_legs_b:
                backup_legs = shortest_backup[1][:]
                backup_legs.append(leg)
                backup = (transfer_leg,backup_legs)
                b_reliability = backup_itinerary_reliability(shortest_next_itinerary[0],backup,start_time,time_budget)
                backup_full = (transfer_leg,backup_legs,b_reliability)
                b_duration = travel_time(backup_legs,start_time)
                # maybe compare to MRIB rel
                if b_reliability > MRB_reliability and timedelta(seconds=0) < b_duration <= time_budget:
                    LIST_Backups.append(backup_full)
                   

    # end of while len(LIST_Backups) > 0 loop       
    print("MRB", MRB)
    return MRB,MRB_reliability
                 
            
# end of while len(LIST_Backups) > 0 loop     

In [23]:
def find_path(origin_node: int,destination_node: int, start_datetime : str, time_budget: timedelta):
    
    '''Initial setup'''
    start_date, start_time = start_datetime.split()

    filtered_legs = filter_network(start_time,start_date,time_budget)
    MRIB_reliability = 0.0
    MRIB = None
    LISTofTRIPS = []
    
    n = 0


    ''' Search initial adjacent legs'''
    filtered_adj_legs = search_adjecent_legs(origin_node,start_time,filtered_legs)
    for leg in filtered_adj_legs:
        itinerary  = [leg]
        reliability = primary_itinerary_reliability(itinerary,start_time,time_budget)
        duration = travel_time(itinerary,start_time)
        expected_arrival_time = leg[4]
        Backups = []
        if reliability > 0 and timedelta(seconds=0) < duration <= time_budget:
            LISTofTRIPS.append([itinerary,reliability,duration,expected_arrival_time,Backups])
        
    #print("visited stops", visited_stops)
    #print(len(LISTofTRIPS))
    end = 0
    '''Main Loop'''
    while LISTofTRIPS:
        
        MRB = None
        MRB_departure_time = None  # To store the MRB's departure time
        first_iteration = True 
        n +=1 #just count
        print("New iteration", n)

        
        passed_stops_n = [origin_node]

        # Find the shortest itinerary(no, min function is not working)
        min_index = find_min_index(LISTofTRIPS)
        shortest_path = LISTofTRIPS.pop(min_index)
        print("Shortest_path",shortest_path)

        tail = shortest_path[0][-1] # last leg of trip
        
        # Check if destination is reached
        if tail[3] == destination_node:
            end += 1
            print("Reached_dest", end)
            MRIB_reliability, MRIB = check_and_update_mrib(shortest_path, MRIB_reliability, MRIB, start_time, time_budget)
            continue

        for leg in shortest_path[0]:
            destination_stop = leg[3]
            passed_stops_n.append(destination_stop)
        


        next_legs = search_adjecent_legs(tail[3],tail[4],filtered_legs)

        # no passed stops!
        next_legs = [leg for leg in next_legs if leg[3] not in passed_stops_n]
        
        next_legs = [
            leg for leg in next_legs 
            if leg[0] == tail[0] or datetime.strptime(leg[2], "%H:%M:%S") >= (datetime.strptime(tail[4], "%H:%M:%S") + timedelta(minutes=2))
        ]
        
        for leg in next_legs:
            itinerary = shortest_path[0] + [leg] #combine previous legs and adjecent
            Backups = shortest_path[4][:] #transfer the backups
            reliability = itinerary_reliability(itinerary,Backups,start_time,time_budget)
            duration = travel_time(itinerary,start_time)
            expected_arrival_time = leg[4]
            trip = [itinerary,reliability,duration,expected_arrival_time,Backups]
            # maybe compare to MRIB rel
            if reliability > 0 and timedelta(seconds=0) < duration <= time_budget: # check initially so we dont waste time later
                print(itinerary)

                if is_transfer(trip[0]) is False: 
                    LISTofTRIPS.append(trip)
                    #print("direct_connecation, main list appended")
                else:
                    '''Backup loop'''
                    #print('Transfer, search for backup')

                    # the backup is stored and reused.
                    if first_iteration or (
                        MRB_departure_time and 
                        MRB_departure_time <= trip[0][-1][2]
                    ):
                        print("HERE",MRB_departure_time, trip[0][-1][2])     
                        if next_legs.index(leg) + 1 < len(next_legs):
                            print("New backup search")
                            MRB, MRB_reliability = backup_search(shortest_path,trip,destination_node,start_time,time_budget,filtered_legs)
                            if MRB:
                                MRB_departure_time = MRB[1][0][2]
                        else:
                            MRB = None
                        
                    if MRB :
                        trip[4] = trip[4][:]
                        trip[4].append(MRB)
                        #print("Appended backup", shortest_next_itinerary)
                    #else:
                        #print("no reliable backup")

                    next_reliability = itinerary_reliability(trip[0],trip[4],start_time,time_budget)
                    if next_reliability >= MRIB_reliability:
                        LISTofTRIPS.append(trip)
                

                #print("no feasible connection, next new path")

             
    return MRIB_reliability, MRIB

In [ ]:
origin_node = "Wien Blumental" #"Laa/Thaya Bahnhof" #"Schattendorf Kirchengasse" #"Gmunden Bahnhof" # for now the departure node is 1
destination_node = "Wien Rennweg" #"Wien Leopoldau" #"Flughafen Wien Bahnhof" #Prinzersdorf Bahnhof"
start_time = "2024-12-13 17:00:00"
time_budget = timedelta(hours=1, minutes = 0)

MRIB_reliability, MRIB = find_path(origin_node,destination_node,start_time,time_budget)

In [28]:
origin_node = "Wien Leopoldau" #"Wien Hauptbahnhof" #"Gmunden Bahnhof" # for now the departure node is 1
destination_node = "Laa/Thaya Bahnhof" #"Laa/Thaya Bahnhof" #Prinzersdorf Bahnhof"
start_time = "2024-12-12 9:00:00"
time_budget = timedelta(hours=1, minutes = 30)

MRIB_reliability, MRIB = find_path(origin_node,destination_node,start_time,time_budget)

1900-01-01 10:30:00
Network 11313
New iteration 1
Shortest_path [[('206.TA.1-S1-W-j24-1.64.R', 'Wien Leopoldau', '09:09:00', 'Wien Süßenbrunn', '09:11:00', '1-S1-W-j24-1', 'TA+n4')], 1, datetime.timedelta(seconds=660), '09:11:00', []]
[('206.TA.1-S1-W-j24-1.64.R', 'Wien Leopoldau', '09:09:00', 'Wien Süßenbrunn', '09:11:00', '1-S1-W-j24-1', 'TA+n4'), ('103.TA.20-SV9-A-j24-1.15.H', 'Wien Süßenbrunn', '10:17:00', 'Deutsch-Wagram Bahnhof', '10:28:00', '20-SV9-A-j24-1', 'TA+oh810')]
HERE None 10:17:00
New backup search
New backup search
MRB None
[('206.TA.1-S1-W-j24-1.64.R', 'Wien Leopoldau', '09:09:00', 'Wien Süßenbrunn', '09:11:00', '1-S1-W-j24-1', 'TA+n4'), ('116.TA.1-S1-W-j24-1.28.R', 'Wien Süßenbrunn', '09:42:00', 'Deutsch-Wagram Bahnhof', '09:47:00', '1-S1-W-j24-1', 'TA+1vj10')]
HERE None 09:42:00
New backup search
New backup search
MRB None
[('206.TA.1-S1-W-j24-1.64.R', 'Wien Leopoldau', '09:09:00', 'Wien Süßenbrunn', '09:11:00', '1-S1-W-j24-1', 'TA+n4'), ('156.TA.2-RX1-W-j24-1.66.R'

KeyboardInterrupt: 

In [26]:
print(MRIB_reliability,MRIB)

1 [[('58.TA.2-RX2-W-j24-1.31.H', 'Laa/Thaya Bahnhof', '09:09:00', 'Kottingneusiedl Bahnhof', '09:13:00', '2-RX2-W-j24-1', 'TA+n4'), ('58.TA.2-RX2-W-j24-1.31.H', 'Kottingneusiedl Bahnhof', '09:13:00', 'Staatz Bahnhof', '09:15:00', '2-RX2-W-j24-1', 'TA+n4'), ('58.TA.2-RX2-W-j24-1.31.H', 'Staatz Bahnhof', '09:16:00', 'Enzersdorf bei Staatz Bahnhof', '09:19:00', '2-RX2-W-j24-1', 'TA+n4'), ('58.TA.2-RX2-W-j24-1.31.H', 'Enzersdorf bei Staatz Bahnhof', '09:19:00', 'Frättingsdorf Bahnhof', '09:23:00', '2-RX2-W-j24-1', 'TA+n4'), ('58.TA.2-RX2-W-j24-1.31.H', 'Frättingsdorf Bahnhof', '09:24:00', 'Hörersdorf Bahnhof', '09:27:00', '2-RX2-W-j24-1', 'TA+n4'), ('58.TA.2-RX2-W-j24-1.31.H', 'Hörersdorf Bahnhof', '09:27:00', 'Siebenhirten (NÖ) Haltestelle', '09:29:00', '2-RX2-W-j24-1', 'TA+n4'), ('58.TA.2-RX2-W-j24-1.31.H', 'Siebenhirten (NÖ) Haltestelle', '09:29:00', 'Mistelbach Stadt', '09:34:00', '2-RX2-W-j24-1', 'TA+n4'), ('58.TA.2-RX2-W-j24-1.31.H', 'Mistelbach Stadt', '09:35:00', 'Mistelbach Bahnho

In [27]:
print(transform_route_info(MRIB,MRIB_reliability))

  🚆 Laa/Thaya Bahnhof (Abfahrt: 09:09:00) → Kottingneusiedl Bahnhof (Ankunft: 09:13:00) → Staatz Bahnhof (Ankunft: 09:15:00) → Enzersdorf bei Staatz Bahnhof (Ankunft: 09:19:00) → Frättingsdorf Bahnhof (Ankunft: 09:23:00) → Hörersdorf Bahnhof (Ankunft: 09:27:00) → Siebenhirten (NÖ) Haltestelle (Ankunft: 09:29:00) → Mistelbach Stadt (Ankunft: 09:34:00) → Mistelbach Bahnhof (Ankunft: 09:36:00) → Wolkersdorf Bahnhof (Ankunft: 09:58:00) → Obersdorf Bahnhof (Ankunft: 10:03:00) → Wien Leopoldau (Ankunft: 10:13:00) mit Linie 2-RX2-W-j24-1

🎯 Endstation: 10:13:00 (Ankunft: 10:13:00)
🔹 Gesamt-Zuverlässigkeit der Route: 1.00

None
